# Project Report - Model

In [1]:
from pathlib import Path
from torch.utils.data import DataLoader

from tqdne.wgan_model import WDiscriminator, WGenerator
from tqdne.gan import WGAN
from tqdne.metric import PowerSpectralDensity, RepresentationInversion, SamplePlot
from tqdne.representations import CenteredMaxEnvelope
from tqdne.utils import load_model, get_last_checkpoint
from tqdne.dataset import WaveformDataset
from tqdne.conf import Config
from tqdne.training import get_pl_trainer

config = Config()

/users/fmachado/.cache/pypoetry/virtualenvs/tqdne-h7mN2Kd5-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/users/fmachado/.cache/pypoetry/virtualenvs/tqdne-h7mN2Kd5-py3.11/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Introduction

**Generative Adversarial Networks (GANs)** have emerged as a groundbreaking paradigm in the field of deep learning, offering a powerful framework for generating realistic data samples from complex probability distributions. Introduced by Ian Goodfellow and his colleagues in 2014, GANs consist of two neural networks, namely the generator and the discriminator, engaged in a minimax game. The generator aims to produce synthetic data samples that are indistinguishable from genuine data, while the discriminator attempts to differentiate between real and fake samples. Through adversarial training, GANs learn to generate highly realistic data across various domains, including images, text, audio, and more.

**Wasserstein GAN (WGAN)** represents a notable advancement in the GAN architecture. Unlike traditional GANs that rely on the Jensen-Shannon divergence or the Kullback-Leibler divergence for training, WGAN introduces the Wasserstein distance as the metric for measuring the discrepancy between the distributions of real and generated data. By optimizing the Wasserstein distance, WGANs offer improved stability and convergence properties compared to their predecessors, mitigating issues such as mode collapse and vanishing gradients commonly associated with traditional GAN training.

In this project report, we delve into the application of Generative Adversarial Networks, particularly Wasserstein GANs, in the context of earthquake seismic wave simulation. By harnessing the capabilities of GANs and WGANs, we aim to develop models capable of generating synthetic seismic waveforms with high statistical resemblence to real-world observations.

## Model Design

### Generator and Discriminator

We start by initializing the Generator and Discriminator of our GAN as below. We use four convolutional layers for both, with the Generator having and embedding layer for its latent space before the convolutional layers and the Discriminator having a fully connected layer after the convolutions to obtain its score.

For the Generator each convolution followed by ReLU and a Batch Normalization layer. 

For the Discriminator each convolution is followed by LeakyReLU.

The classes that implement the model are on `tqdne.wgan_model`

In [2]:
G = WGenerator(
    latent_dim=128,
    wave_size=1024,
)
print(G)


WGenerator(
  (latent_to_features): Sequential(
    (0): Linear(in_features=144, out_features=16384, bias=True)
    (1): ReLU()
  )
  (features_to_signal): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (4): ReLU()
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (7): ReLU()
    (8): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose1d(32, 1, kernel_size=(4,), stride=(2,), padding=(1,))
  )
)


In [3]:
D = WDiscriminator(
    wave_size=1024,
)
print(D)

WDiscriminator(
  (image_to_features): Sequential(
    (0): Conv1d(1, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv1d(32, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.2)
    (4): Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.2)
    (6): Conv1d(128, 256, kernel_size=(4,), stride=(2,), padding=(1,))
    (7): LeakyReLU(negative_slope=0.2)
  )
  (features_to_prob): Sequential(
    (0): Linear(in_features=16400, out_features=1, bias=True)
  )
)


### Embedding of Conditions:

To account for the fact that we want to generate seismic data under certain conditions, namely the distance from the hypocenter and the magnitude, we have to add conditional information to our model. 

In particular, every waveform has a single scalar as `dist` and one as `mag`. We use positional encoding to add this information both to the Generator and Discriminator, although in slightly different ways.

- *Generator:* We concatenate the positional embedding of the conditions with the latent space input of the Generator. This means the embedding goes through the fully connected and convolutional layers alongside the random sample from the latent space.

- *Discriminator:* We concatenate the positional embedding of the conditions to the output of the block of convolutional layers. This means the conditions are only considered at the final fully connected layer of the discriminator.

Both Generator and Discriminator accept a hyperparameter `encoding_L` to be the length used by the positional encoding.

For a complete set of hyperparameters, here it follows:

In [4]:
generator_parameters = {
    "latent_dim": 128,
    "wave_size": 1024,
    "out_channels": 2, #Depends on the representation chosen
    "encoding_L": 8,
    "num_cond_vars": 2, #Mag and Dist
    "dim": 32, #Controls the size of the convolution layers
}
discriminator_parameters = {
    "wave_size": 1024,
    "in_channels": 2, #Depends on the representation chosen
    "encoding_L": 8,
    "num_cond_vars": 2, #Mag and Dist
    "dim": 32, #Controls the size of the convolution layers
}

### WGAN

The Wasserstein GAN implemented in `tqdne.gan` is a PyTorch Lightning Module that initializates both Generator and Discriminator and coordinates the training. It takes care of the training, sampling, and all the interactions with the model. We use the Adam optimizer throughout the experiments.

It notably has a common modification to GAN training by running one step of the Generator for each `n_critics` steps for the Discriminator, where `n_critics` is a hyperparameter. Moreover, we also have a hyperparameter `reg_lambda` that controls the multiplicative constant of the gradient penalty of the WGAN loss.

For a full set of hyperparameters, refer below:

In [5]:
optimizer_parameters = {
    "lr": 1e-4,
    "b1": 0.9,
    "b2": 0.99,
}
model_parameters = {
    "reg_lambda": 10.0,
    "n_critics": 3,
    "optimizer_params": optimizer_parameters,
    "generator_params": generator_parameters,
    "discriminator_params": discriminator_parameters,
    "conditional": True, # Flag to use conditional WGAN
}

In [6]:
model = WGAN(**model_parameters)

2024-02-12 23:07:25 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmpmjqal04j
2024-02-12 23:07:25 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmpmjqal04j/_remote_module_non_scriptable.py


## Runs & Metrics

Due to the limited performance of the approach so far, we limited the metrics of performance to visual inspection and negative critic loss, the latter being considered as a good indicator of convergence of a GAN.

The metrics are automatically tracked on Weights & Biases, which is from where we take some of the visualizations below.

Metrics inherit from a standard interface `AbstractMetric` and allow for simple modularity. Each metric implements a update, compute and plot method, to obtain new data, compute the metric, and possibly plot it, respectively.

More importantly, we heavily use a wrapper metric `RepresentationInversion` that is responsible for inverting the `Representation` transformation used for the dataset before computing or plotting the desired metric.

Indeed, here it follows an example of how we use the metrics:

In [7]:
representation = CenteredMaxEnvelope(config)
metrics = [PowerSpectralDensity(config.fs, channel=0), SamplePlot(config.fs, channel=0)]
metrics = [RepresentationInversion(m, representation) for m in metrics]

Now we also include the dataset, as it will be necessary for what comes next.

In [8]:
train_path = config.datasetdir / Path(config.data_upsample_train)
test_path = config.datasetdir / Path(config.data_upsample_test)
dataset_train = WaveformDataset(train_path, representation, reduced=1024)
dataset_test = WaveformDataset(test_path, representation, reduced=1024)

Now it only remains to load the Pytorch Lighning trainer and start it. Indeed, we have a helper function `get_pl_trainer` which configures and retuns our trainer but also attaches the logic for the handling of metrics to the trainer.
It goes as following:

In [9]:
trainer_parameters = {
    "max_epochs": 100,
    "accelerator": "auto",
    "devices": "auto",
    "log_every_n_steps": 10,
}
trainer = get_pl_trainer(
    name="Interactive WGAN",
    val_loader=DataLoader(dataset_test, batch_size=64),
    metrics=metrics,
    eval_every=10,
    log_to_wandb=True,
    config=config,
    **trainer_parameters,
)

/users/fmachado/.cache/pypoetry/virtualenvs/tqdne-h7mN2Kd5-py3.11/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /users/fmachado/.cache/pypoetry/virtualenvs/tqdne-h7 ...
2024-02-12 23:07:38 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: False, used: False
2024-02-12 23:07:38 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2024-02-12 23:07:38 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2024-02-12 23:07:38 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs


Now it remains to start training with the fit function, with the option of resuming the training that is automatically stored at the configured output directory.

In [ ]:
resume = False

if resume:
    last_checkpoint = get_last_checkpoint(config.outputdir / "Interactive WGAN")
else:
    last_checkpoint = None

# trainer.fit(
#     model,
#     train_dataloaders=DataLoader(dataset_train, batch_size=64),
#     test_dataloaders=DataLoader(dataset_test, batch_size=64),
#     ckpt_path=last_checkpoint,
# )

Optionally, we can just load the last checkpoint as done below:

In [10]:
checkpoint = get_last_checkpoint(config.outputdir / "WGAN")
model = WGAN.load_from_checkpoint(checkpoint)

2024-02-12 23:07:38 - root - INFO - Last checkpoint is : /users/fmachado/tqdne/outputs/WGAN/name=0_epoch=47-val_loss=-24.74.ckpt
